In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import re
from openpyxl import Workbook, load_workbook

In [2]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\alsoj\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


In [9]:
URL = 'http://kcna.kp/kp/category/articles/q/1ee9bdb7186944f765208f34ecfb5407.kcmsf'

In [10]:
browser.get(URL)

In [15]:
article_wrapper = browser.find_element(by=By.CLASS_NAME, value='article-link')

In [16]:
a_tag = article_wrapper.find_element(by=By.TAG_NAME, value='a')

In [17]:
a_tag.click()

In [3]:
# 엑셀 생성
wb = Workbook()
ws = wb.active

# 제목 적기
sub = ['식별키','목록-번호','목록-구분','목록-제목','목록-면수','목록-작성자','목록-일자','본문-일자(원본)','본문-일자','본문-내용','본문-제목','본문-부제(제목앞)','본문-부제(제목뒤)']
for kwd, j in zip(sub, list(range(1, len(sub)+1))):
    ws.cell(row=1, column=j).value = kwd

wb.save('./output/노동신문.xlsx')

In [4]:
list_base_url = 'http://www.rodong.rep.kp/ko/index.php?strPageID=SF01_01_03&strDate='
date = '2022-06-01'

In [5]:
browser.get(list_base_url+date)

### 크롤링 범위
- 목록-번호 / 목록-구분 / 목록-제목 / 목록-면 / 목록-작성자 / 목록-일자
- 본문-일자(원본) / 본문-일자 / 본문-내용 / 본문-제목 / 본문-부제(제목앞) / 본문-부제(제목뒤)

- '식별키','목록-번호','목록-구분','목록-제목','목록-면수','목록-작성자','목록-일자','본문-일자(원본)','본문-일자','본문-내용','본문-제목','본문-부제(제목앞)','본문-부제(제목뒤)'

## 뉴스 목록 크롤링

In [6]:
news_line_list = browser.find_elements(by=By.CLASS_NAME, value='ListNewsLineContainer')

In [7]:
def get_article_info(title_full) :
    if title_full.startswith('[') :
        article_title = title_full[title_full.find(']')+1 : title_full.rfind('[')].strip()
        article_type = title_full[title_full.find('[')+1 : title_full.find(']')].strip()
        article_page = title_full[title_full.rfind('[')+1 : title_full.rfind(']')].strip()
    else :
        article_title = title_full[ : title_full.rfind('[')].strip()
        article_type = ''
        article_page = title_full[title_full.rfind('[')+1 : title_full.rfind(']')].strip() 
    
    return article_title, article_type, article_page

In [67]:
for news_line in news_line_list:
    temp_row = []
    
    line_no = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineNo').text.strip()
    line_title = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW').text.strip()
    line_writer = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineWriter').text.strip()
    line_date = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineDate').text.strip()
    
    article_title, article_type, article_page = get_article_info(line_title)

    temp_row.append(line_no)
    temp_row.append(article_type)
    temp_row.append(article_title)
    temp_row.append(article_page)
    temp_row.append(line_writer)
    temp_row.append(line_date)
    
    # 상세창 팝업
    title_popup = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW')
    title_popup_a = title_popup.find_element(by=By.TAG_NAME, value='a')
    title_popup_a.click()    
    browser.switch_to.window(browser.window_handles[1])
    
    key = browser.current_url[-15:]
    temp_row.insert(0, key)
    
    detail_date_origin = browser.find_element(by=By.CLASS_NAME, value='ArticleMenuDate').text
    temp_row.append(detail_date_origin)
    
    detail_date = key[:10]
    temp_row.append(detail_date)
    
    detail_title = ''
    detail_sub_title = ''
    detail_content = ''
    
    detail_content_list = browser.find_elements(by=By.CLASS_NAME, value='ArticleContent')

    for detail_line in detail_content_list:
        title = detail_line.find_elements(by=By.TAG_NAME, value='font')

        if title and title[0].text.strip() != '' :
            if 'bold' in title[0].get_attribute('style') :
                detail_title += title[0].text + ' '
            else :
                detail_sub_title += title[0].text + ' '
        elif detail_line.text.strip() != '' :
            detail_content += ' ' + detail_line.text
    
    temp_row.append(detail_content)
    temp_row.append(detail_title.strip())
    temp_row.append(detail_sub_title.strip())
    temp_row.append(detail_sub_title.strip())
    ws.append(temp_row)
    wb.save('./output/노동신문.xlsx')
    
    # 팝업창 close 후 객체 전환
    browser.close() 
    browser.switch_to.window(browser.window_handles[0])

In [16]:
title_popup = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW')
title_popup_a = title_popup.find_element(by=By.TAG_NAME, value='a')
title_popup_a.click()

In [51]:
browser.window_handles

['CDwindow-2FACECDB0A3970DC7739F128E93F439C',
 'CDwindow-6148C768679F565202DC9F8ABABF7B45',
 'CDwindow-13C5389092528A7700BC5A53049A293B']

In [8]:
browser.switch_to.window(browser.window_handles[-1])

In [9]:
key = browser.current_url[-15:]
key

'2022-06-01-0008'

## 뉴스 상세 크롤링

In [10]:
detail_date_origin = browser.find_element(by=By.CLASS_NAME, value='ArticleMenuDate').text
detail_date_origin

'주체111(2022)년 6월 1일 수요일'

In [11]:
detail_date = key[:10]
detail_date

'2022-06-01'

In [18]:
detail_title = ''
detail_sub_title = ''
detail_content = ''

In [19]:
detail_content_list = browser.find_elements(by=By.CLASS_NAME, value='ArticleContent')

for detail_line in detail_content_list:
    title = detail_line.find_elements(by=By.TAG_NAME, value='font')
    
    if title and title[0].text.strip() != '' :
        if 'bold' in title[0].get_attribute('style') :
            detail_title += title[0].text    
        else :
            detail_sub_title += title[0].text    
    elif detail_line.text.strip() != '' and 'right' not in detail_line.get_attribute('style') :        
        detail_content += detail_line.text + '\r\n'

In [20]:
detail_content

'경애하는 김정은동지께서는 다음과 같이 말씀하시였다.\r\n《전군의 모든 지휘관, 병사들은 인민군대가 인민을 떠나서는 한시도 존재할수 없음을 항상 명심하고 인민을 무한히 존중하고 사랑하며 인민을 위함이라면 사선도 헤치는 혁명군대의 훌륭한 풍모를 계속 견지해나가야 하겠습니다.》\r\n수도의 평범한 약국들이 조선인민군 군의부문 전투원들이 지켜선 인민사수전의 전호로 된 그때로부터 어느덧 보름 남짓이 흘렀다.\r\n성스러운 우리 당마크가 새겨진 붉은 파견장을 전선파견장, 특별신임장으로 간직한 인민군대의 군의부문 전투원들은 평양시안의 수백개의 약국들에 긴급전개한 그 순간부터 지금까지 방역전장마다에서 인민사랑의 서사시를 끊임없이 수놓아가고있다.\r\n얼마전 평천구역 새마을２동 ２７인민반의 한 살림집에서는 눈물없이는 볼수 없는 감동깊은 화폭이 펼쳐졌다.\r\n《군관동지, 우리 인민반주민을 위해 꼬박 밤을 밝히며 치료를 해주어 정말 고맙습니다.》\r\n홀로 사는 늙은이여서 이웃들도 관심을 많이 돌려왔지만 갑자기 그가 앓아누웠을 때 기울이는 인민군대의 정성은 그보다 더 뜨거웠다.\r\n인민반장이 허리굽혀 인사하려 하자 남송상점 평천분점 의약품매대에서 온 군관은 황황히 그를 만류하며 우리 군대는 다 인민의 아들딸이라고, 자식이 부모가 앓을 때 효도를 하는것이야 너무도 응당한 일이 아닌가고 하면서 이제부터는 홀로 사는 로인을 친어머니로 모시겠다고 하는것이였다.\r\n혈연의 정이 마디마디 넘쳐흐르는 군관의 말에 인민반장과 집주인도 울고 이웃들도 울었다.\r\n강동군 태잠리약국의 판매원과 리진료소의 의료일군들이 그곳에 파견된 화선용사들에 대하여 지금도 계속 외우는 말이 있다.\r\n《며칠전 우리 약국에 파견된 군의동지들중 두명이 갑자기 앓아눕게 되였습니다.그래서 약을 가져다주니 인민들에게 공급되는 약은 한알도 다치면 안된다고 하면서 자체로 고려치료를 하는것이였습니다.》\r\n인민을 위해서는 수십리 먼곳에까지 이동봉사를 나가 사랑의 불사약을 안겨주면서도 자기들이 앓을 때에는 단 한알의

In [64]:
# 팝업창 종료 후 close
browser.close() 
browser.switch_to.window(browser.window_handles[0])